In [ ]:
import pandas as pd
import os 
import re
import nltk
from nltk.tokenize import word_tokenize
import boto3
from langdetect import detect
from collections import Counter
from process_helper import * 

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
news = pd.read_excel('news_excerpts_parsed.xlsx')
leaks = pd.read_excel('wikileaks_parsed.xlsx')

In [ ]:
leaks['Text'] = leaks['Text'].astype(str)
news['Text'] = news['Text'].astype(str)

In [ ]:
news.info()

In [ ]:
leaks

In [ ]:
leaks['Text'][1]

In [ ]:
def check_for_patterns(df, patterns):
    results = {}
    for pattern in patterns:
        contains_pattern = df['Text'].str.contains(pattern, na=False)
        results[pattern] = contains_pattern.sum()  # Count rows containing the pattern
    return results

# Patterns to check for
patterns_to_check = ['\n', '\xad', '\t', '�', r'\\']

# Check both datasets for the patterns
print(f"News : {check_for_patterns(news, patterns_to_check)}")
print(f"Leaks : {check_for_patterns(leaks, patterns_to_check)}")

In [ ]:
leaks['Text'] = leaks['Text'].str.replace('\n', '.', regex=True)
news['Text'] = news['Text'].str.replace('\n', '.', regex=True)
leaks['Text'] = leaks['Text'].str.replace('\xad', '-', regex=True)
leaks['Text'] = leaks['Text'].str.replace( r'\\', "''", regex=True)

In [ ]:
leaks

In [ ]:
leaks['Text'][52]

In [ ]:
leaks = leaks.groupby('PDF Path', as_index=False).agg({
    'Text': ' '.join  
})

In [ ]:
leaks

In [ ]:
leaks['Text'][0]

In [ ]:
leaks['Token_Count'] = leaks['Text'].apply(lambda x: len(word_tokenize(x)))

In [ ]:
news['Token_Count'] = news['Text'].apply(lambda x: len(word_tokenize(x)))

In [ ]:
leaks

In [ ]:
news

In [ ]:
print('inv' if any(news['Token_Count'] > 300) else 'ok')
print('inv' if any(leaks['Token_Count'] > 4000) else 'ok')

In [ ]:
language_counts_news = Counter(news['Text'].apply(detect))
language_counts_leaks = Counter(leaks['Text'].apply(detect))
print(language_counts_news)
print(language_counts_leaks)

In [ ]:
news

In [ ]:
news.drop(columns=['Link','Token_Count'], inplace= True)

In [ ]:
leaks.drop(columns=['PDF Path','Token_Count'], inplace= True)

In [ ]:
news.to_csv('news.csv', index=False)
leaks.to_csv('leaks.csv',index=False)

In [ ]:
"""
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  # For newer versions
import pyLDAvis.gensim_models
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Transform text into a document-term matrix
vectorizer = CountVectorizer(stop_words='english', max_features=5000)
doc_term_matrix = vectorizer.fit_transform(news['Text'])

# Fit an LDA model
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(doc_term_matrix)

# Visualization for pyLDAvis
pyLDAvis.enable_notebook()

# Prepare the visualization
lda_vis = pyLDAvis.prepare(
    topic_term_dists=lda_model.components_,
    doc_topic_dists=lda_model.transform(doc_term_matrix),
    doc_lengths=[len(doc.split()) for doc in news['Text']],
    vocab=vectorizer.get_feature_names_out(),
    term_frequency=doc_term_matrix.sum(axis=0).A1
)

pyLDAvis.display(lda_vis)
"""

In [ ]:
"""
# Create a document-term matrix for the 'leaks' dataset
leaks_term_matrix = vectorizer.fit_transform(leaks['Text'])

# Fit an LDA model using the 'leaks' dataset
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(leaks_term_matrix)

# Visualization for pyLDAvis
pyLDAvis.enable_notebook()

# Prepare the visualization
lda_vis = pyLDAvis.prepare(
    topic_term_dists=lda_model.components_,
    doc_topic_dists=lda_model.transform(leaks_term_matrix),
    doc_lengths=[len(doc.split()) for doc in leaks['Text']],
    vocab=vectorizer.get_feature_names_out(),
    term_frequency=leaks_term_matrix.sum(axis=0).A1
)

pyLDAvis.display(lda_vis)
"""

In [ ]:
upload_files_to_s3('bia-datathon-text-data-unstructured', folder_path)